In [21]:
import copy
from dataclasses import dataclass, field
from enum import IntEnum
from typing import TypeVar, Sequence, Optional, List, Iterator, Union

import clingo
import clingox.program

from starlingo.Atom import Atom
from starlingo.Literal import BasicLiteral, Literal
from starlingo.Program import from_string, Program
from starlingo.Rule import Constraint, NormalRule
from starlingo.Symbol import Function, Term, IntegerConstant


In [22]:
program = """
% F
fluent(m).
fluent(ab).
% D
default(ab).
% E
action(d).
% AD
impossible(d, m, -ab).
% Aleph
occ_at(d, 1).
"""
ps = from_string(program)
p_raw = ps[0]

In [23]:
print(p_raw.custom_str(sep='\n'))

#program base.
fluent(m).
fluent(ab).
default(ab).
action(d).
impossible(d,m,-ab).
occ_at(d,1).


In [24]:
__t = Function('__t')

In [25]:
from starlingo.Symbol import Variable

signature = []
new_rules = []
for rule in p_raw.rules:
    if rule.head.atom.symbol.name == 'impossible':
        impossible_if_stm: Function = rule.head.atom.symbol
        new_rules.append(
            Constraint((BasicLiteral(Atom(Function('occ_at', (impossible_if_stm.arguments[0], __t)))),
                        *(BasicLiteral(Atom(Function('obs_at', (argument, __t)))) for argument in
                          impossible_if_stm.arguments[1:]))))
    elif rule.head.atom.symbol.name == 'if':
        if_stm: Function = rule.head.atom.symbol
        head = BasicLiteral(Atom(Function('obs_at', (if_stm.arguments[0], __t))))
        body = tuple(
            BasicLiteral(Atom(Function('obs_at', (argument, __t)))) for argument in if_stm.arguments[1:])
        new_rules.append(NormalRule(head, body))
    elif rule.head.atom.symbol.name == 'causes':
        causes_stm: Function = rule.head.atom.symbol
        head = BasicLiteral(Atom(Function('obs_at', (
            causes_stm.arguments[1], __t + Term(IntegerConstant(1))))))
        body = (BasicLiteral(Atom(Function('occ_at', (causes_stm.arguments[0], __t)))),
                *(BasicLiteral(Atom(Function('obs_at', (argument, __t)))) for argument in
                  causes_stm.arguments[2:]))
        new_rules.append(NormalRule(head, body))
    else:
        signature.append(rule)

new_rules.append(
    NormalRule(
        BasicLiteral(Atom(Function('obs_at', (
            Variable('F'), __t + Term(IntegerConstant(1)))))),
        (
            BasicLiteral(Atom(Function('obs_at', (Variable('F'), __t)))),
            -BasicLiteral(Atom(Function('obs_at', (-Variable('F'), __t + Term(IntegerConstant(1)))))),
            -BasicLiteral(Atom(Function('obs_at', (Function('u', (Variable('F'),)), __t + Term(IntegerConstant(1)))))),
        ),
    )
)

new_rules.append(
    NormalRule(
        BasicLiteral(Atom(Function('obs_at', (-Variable('F'), __t + Term(IntegerConstant(1)))))),
        (
            -BasicLiteral(Atom(Function('obs_at', (Variable('F'), __t + Term(IntegerConstant(1)))))),
            -BasicLiteral(Atom(Function('obs_at', (Function('u', (Variable('F'),)), __t + Term(IntegerConstant(1)))))),
            BasicLiteral(Atom(Function('obs_at', (-Variable('F'), __t)))),
        )
    )
)

new_rules.append(
    NormalRule(
        BasicLiteral(Atom(Function('obs_at', (Function('u', (Variable('F'),)), __t + Term(IntegerConstant(1)))))),
        (
            -BasicLiteral(Atom(Function('obs_at', (Variable('F'), __t + Term(IntegerConstant(1)))))),
            BasicLiteral(Atom(Function('obs_at', (Function('u', (Variable('F'),)), __t)))),
            -BasicLiteral(Atom(Function('obs_at', (-Variable('F'), __t + Term(IntegerConstant(1)))))),
        )
    )
)
new_rules.append(
    NormalRule(
        BasicLiteral(Atom(Function('obs_at', (Variable('F'), Term(IntegerConstant(1)))))),
        (
            BasicLiteral(Atom(Function('init', (Variable('F'),)))),
        )
    )
)
new_rules.append(
    NormalRule(
        BasicLiteral(Atom(Function('obs_at', (-Variable('F'), Term(IntegerConstant(1)))))),
        (
            BasicLiteral(Atom(-Function('init', (Variable('F'),)))),
        )
    )
)
new_rules.append(
    NormalRule(
        BasicLiteral(Atom(Function('obs_at', (Variable('F'), Term(IntegerConstant(1)))))),
        (
            BasicLiteral(Atom(Function('forced', (Variable('F'),)))),
            BasicLiteral(Atom(Function('default', (Variable('F'),)))),
            -BasicLiteral(Atom(-Function('init', (Variable('F'),)))),
        )
    )
)
action_language = """
#program action_language(__t).
:- occ_at(d,__t), obs_at(m,__t), obs_at(-ab,__t).
obs_at(F,__t+1) :- obs_at(F,__t), not obs_at(-F,__t+1), not obs_at(u(F),__t+1).
obs_at(-F,__t+1) :- not obs_at(F,__t+1), not obs_at(u(F),__t+1), obs_at(-F,__t).
obs_at(u(F),__t+1) :- not obs_at(F,__t+1), obs_at(u(F),__t), not obs_at(-F,__t+1).
obs_at(F,1) :- init(F).
obs_at(-F,1) :- -init(F).
obs_at(F,1) :- forced(F), default(F), not -init(F).
"""

p = Program(rules=signature)
pa = from_string(action_language)[1]
print(p.custom_str(sep='\n'), pa.custom_str(sep='\n'), sep='\n')

AssertionError: AST __t should have type ASTType.Function, but has type ASTType.Id.